In [23]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('B://workspace/tensorflow/')

from src.main.predict import load_rf_model

rf_model = load_rf_model()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
from numpy import mean

from src.utils.mld import compute_rmse, get_mld
from src.config.constants import deep
from src.plot.mld import plot_mld, plot_mld_rmse
from src.plot.sst import plot_sst_rmses
from src.plot.profile import plot_profile_rmses, plot_3d_temperature_comparison
from src.config.params import Areas, Area
from src.main.predict import predict_sst, predict_rf, load_sst_model

rf_months = [239, 237, 234, 230]
dates = ['2024-01', '2023-10', '2023-08', '2023-04']

length = len(rf_months)

# 月平均误差
sst_rmses = { }
profile_rmses = {}

global_area = [Area(title=f"global_r1_l4", lon=[-180, 180], lat=[-80, 80], description="Global")]

for area in global_area:
    
    print(f"area: =========================== {area.title} ============================")
    
    sst_model = load_sst_model(area)
    area_sst_rmse = ()
    area_profile_rmse = []
    
    for i in range(length):        
        print(f"time: ===================== {dates[i]} =====================")
        
        rf_month = rf_months[i]

        pred_sst, sst_rmse = predict_sst(sst_model, area, rf_month)
        
        print(f'sst_rmse: {sst_rmse}')
        
        profile, origin_profile, rmses_profile  = predict_rf(rf_model, area, pred_sst, rf_month)
        area_profile_rmse.append(rmses_profile)
        
        print(f"rmses_profile: {rmses_profile}")
        
        # 绘图
        # plot_sst_seq(sst_queue, area.lon, area.lat)
        plot_3d_temperature_comparison(origin_profile, profile, area.lat, area.lon, deep)
        
        # # 混合层
        # mld = get_mld(rf_month, area.lon, area.lat)
        
        # # 基于混合层计算误差
        # mld_rmse = compute_rmse(mld, profile, origin_profile)
        
        # plot_mld_rmse(mld_rmse, area.lon, area.lat, f"{dates[i]}-{area.title} mld rmse")
        
        # plot_mld(mld, area.lon, area.lat)
    
    sst_rmses[area.title] = area_sst_rmse
    profile_rmses[area.title] = area_profile_rmse

plot_profile_rmses(profile_rmses, dates)



area: =========================== Area a ============================
time: ===================== 2024-01 =====================
起始时间：2023-12-01
torch.Size([1, 1, 1, 10, 10])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x100 and 14400x1024)